### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [108]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [116]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [109]:
# Calcualte different measures and then assign to variables
total_schools = len(school_data_complete["school_name"].unique())
total_students = len(school_data_complete["Student ID"].unique())
total_budget = school_data_complete["budget"].unique().sum()
average_math_score = school_data_complete["math_score"].mean()
average_reading_score = school_data_complete["reading_score"].mean()
percent_passing_math = school_data_complete.loc[school_data_complete["math_score"]>=70,"math_score"].count()/total_students*100
percent_passing_reading = school_data_complete.loc[school_data_complete["reading_score"]>=70,
                                                   "reading_score"].count()/total_students*100
percent_passing_math_reading = school_data_complete.loc[(school_data_complete["math_score"]>=70) & 
                         (school_data_complete["reading_score"]>=70),"Student ID"].count()/total_students*100

# Create summary dataframe to display summary information
summary_df = pd.DataFrame(
    {"Total Schools":total_schools, "Total Students":total_students, 
     "Total Budget": total_budget, "Average Math Score":average_math_score, 
     "Average Reading Score":average_reading_score, "% Passing Math":percent_passing_math,
     "% Passing Reading":percent_passing_reading, "% Overall Passing":percent_passing_math_reading
}, index = [0])

#Format some measures to the desired format
summary_df['Total Students']=summary_df['Total Students'].map("{:,.0f}".format)
summary_df['Total Budget']=summary_df['Total Budget'].map("${:,.2f}".format)
summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [125]:
# Groupby school data by school name
school_summary = school_data_complete.groupby(['school_name','type'])

# Calculate total_budget, total_student, per_student_budget, average_math_score, and average_reading_score by school name
total_budget = school_summary["budget"].mean()
total_student= school_summary['Student ID'].count()
per_student_budget = total_budget/total_student
average_math_score = school_summary['math_score'].mean()
average_reading_score = school_summary['reading_score'].mean()

# Add calcuatlion results above to the school_summary_df dataframe
school_summary_df = pd.DataFrame({"Total Students": total_student,"Total School Budget":total_budget, 
                                  "Per Student Budget":per_student_budget,"AVerage Math Score":average_math_score,
                                  "Average Reading Score":average_reading_score})

# Calcuate percent_pass_math, percent_pass_reading, and percent_pass_overall
# Find the unique list of schools
school_names = school_data_complete["school_name"].unique()      
pass_math_count=[]
pass_reading_count=[]
pass_both_count=[]

# For each school, find the total student count who passes 70 on math, reading, and both
for school in school_names:
    pass_math_count.append(school_data_complete.loc[(school_data_complete['school_name']==school)&(school_data_complete['math_score']>=70),'Student ID'].count())
    pass_reading_count.append(school_data_complete.loc[(school_data_complete['school_name']==school)&(school_data_complete['reading_score']>=70),'Student ID'].count())
    pass_both_count.append(school_data_complete.loc[(school_data_complete['school_name']==school)&(school_data_complete['reading_score']>=70)&(school_data_complete['math_score']>=70),'Student ID'].count())  

# Creating a new datafarme to hold the total students who passes math, reding, and both
passing_count_df = pd.DataFrame({"school_name":school_names, 
                                   "Count_Student_Pass_Math": pass_math_count,
                                    "Count_Student_Pass_Reading":pass_reading_count,
                                    "Pass_both_count":pass_both_count})

# Merge the new dataframe with the school_summary_df to add new columns from passing_count_df
school_summary_df = pd.merge(school_summary_df, passing_count_df, how="left", on=["school_name", "school_name"])

# Calcuate percent_pass_math, percent_pass_reading, and percent_pass_both
percent_pass_math = school_summary_df['Count_Student_Pass_Math']/school_summary_df['Total Students']*100
percent_pass_reading = school_summary_df['Count_Student_Pass_Reading']/school_summary_df['Total Students']*100
percent_pass_both = school_summary_df['Pass_both_count']/school_summary_df['Total Students']*100

# Assiging % pass math, reading and both to the school_summary_df
school_summary_df['% Passing Math'] = percent_pass_math
school_summary_df['% Passing Reading'] = percent_pass_reading
school_summary_df['% Overall Passing'] = percent_pass_both

# Drop the following columns that only for calucation purpose
school_summary_df = school_summary_df.drop(['Count_Student_Pass_Math','Count_Student_Pass_Reading','Pass_both_count'], axis =1)

# Set school name as index
school_summary_df = school_summary_df.set_index('school_name',drop = True)
school_summary_df.index.name = ''

# Formatting 'Total School Budget', 'Per Student Budget'
school_summary_df['Total School Budget'] = school_summary_df['Total School Budget'].map("${:,.2f}".format)
school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget'].map("${:.2f}".format)
school_summary_df

#missing the school type

,Total Students,Total School Budget,Per Student Budget,AVerage Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,
Bailey High School,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [111]:
# sort_values doesn't work...research
school_summary_df = school_summary_df.sort_values("Total School Budget", ascending = False)
school_summary_df.head(5)

,Total Students,Total School Budget,Per Student Budget,AVerage Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,
Griffin High School,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Pena High School,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541
Bailey High School,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Johnson High School,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Hernandez High School,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [112]:
# sort_values doesn't work...research
school_summary_df = school_summary_df.sort_values("% Overall Passing")
school_summary_df.head(5)

,Total Students,Total School Budget,Per Student Budget,AVerage Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,
Rodriguez High School,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [113]:
math_scores_by_grade = school_data_complete.groupby(['school_name','grade'])
math_scores_by_grade.head()
math_avearge = math_scores_by_grade['math_score'].mean()
math_avearge

school_name            grade
Bailey High School     10th     76.996772
                       11th     77.515588
                       12th     76.492218
                       9th      77.083676
Cabrera High School    10th     83.154506
                       11th     82.765560
                       12th     83.277487
                       9th      83.094697
Figueroa High School   10th     76.539974
                       11th     76.884344
                       12th     77.151369
                       9th      76.403037
Ford High School       10th     77.672316
                       11th     76.918058
                       12th     76.179963
                       9th      77.361345
Griffin High School    10th     84.229064
                       11th     83.842105
                       12th     83.356164
                       9th      82.044010
Hernandez High School  10th     77.337408
                       11th     77.136029
                       12th     77.186567
     

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type